In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## Data Preprocessing

In [ ]:
emp_df = pd.read_csv('data/Human_Resources.csv')

In [ ]:
# Drop columns in which all rows have the same values
# Also drop EmployeeNumber which does not contribution to the problem
emp_df.drop(['EmployeeCount', 'StandardHours','Over18','EmployeeNumber'], axis=1,inplace=True)

In [ ]:
emp_df.select_dtypes(include=object).columns.to_list()

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [ ]:
# One Hot Encoding for Categorical Variables
from sklearn.preprocessing import OneHotEncoder
X_cat = emp_df[['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','OverTime']]
ohe = OneHotEncoder()
X_cat = ohe.fit_transform(X_cat).toarray()
X_cat = pd.DataFrame(X_cat)

X_num = emp_df.drop(['BusinessTravel','Department','EducationField','Gender',
                     'JobRole','MaritalStatus','OverTime','Attrition'],axis=1)

In [ ]:
X = pd.concat([X_cat,X_num],axis=1)
y = emp_df['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , train_size = 0.75, stratify = y)

In [ ]:
X_train.shape, X_test.shape

((1102, 51), (368, 51))

In [ ]:
# Normalize independent variables
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import pickle
pickle.dump(ohe, open('model/OneHotEncoder.pickle','wb'))
pickle.dump(scaler, open('mode/MinMaxScaler.pickle','wb'))

In [ ]:
from imblearn.combine import SMOTEENN
smoteenn = SMOTEENN(random_state = 4)
X_train, y_train = smoteenn.fit_resample(X_train, y_train)

In [ ]:
print("Ratio of 'Yes' in Attrition Before SMOTE+ENN : " , round(sum(y)/len(y),2))
print("Ratio of 'Yes' in Attrition After SMOTE+ENN :" , round(sum(y_train)/len(y_train),2))

Ratio of 'Yes' in Attrition Before SMOTE+ENN :  0.16
Ratio of 'Yes' in Attrition After SMOTE+ENN : 0.62


## Logistics Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model_log = LogisticRegression(max_iter=1000)
model_log.fit(X_train, y_train)

y_pred_log = model_log.predict(X_test)

In [ ]:
print(f'Accuracy: {100*accuracy_score(y_pred_log, y_test): .3f}%')
print('\n')
print('Confusion Matrix: \n', confusion_matrix(y_pred_log, y_test))
print('\n')
print('Classification Report: \n', classification_report(y_pred_log, y_test))

Accuracy:  65.761%


Confusion Matrix: 
 [[193  10]
 [116  49]]


Classification Report: 
               precision    recall  f1-score   support

           0       0.62      0.95      0.75       203
           1       0.83      0.30      0.44       165

    accuracy                           0.66       368
   macro avg       0.73      0.62      0.60       368
weighted avg       0.72      0.66      0.61       368



## Random Forest

In [ ]:
from  sklearn.ensemble import RandomForestClassifier

In [ ]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)

y_pred_rf = model_rf.predict(X_test)

In [ ]:
print(f'Accuracy: {100*accuracy_score(y_pred_rf, y_test): .3f}%')
print('\n')
print('Confusion Matrix: \n', confusion_matrix(y_pred_rf, y_test))
print('\n')
print('Classification Report: \n', classification_report(y_pred_rf, y_test))

Accuracy:  79.620%


Confusion Matrix: 
 [[261  27]
 [ 48  32]]


Classification Report: 
               precision    recall  f1-score   support

           0       0.84      0.91      0.87       288
           1       0.54      0.40      0.46        80

    accuracy                           0.80       368
   macro avg       0.69      0.65      0.67       368
weighted avg       0.78      0.80      0.78       368



## Artificial Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model_ann = Sequential()

model_ann.add(Dense(units = 25, activation = 'relu', input_shape = (51,)))
model_ann.add(Dense(units = 25, activation = 'relu'))
model_ann.add(Dropout(0.5))
model_ann.add(Dense(units = 25, activation = 'relu'))
model_ann.add(Dropout(0.5))
model_ann.add(Dense(units = 25, activation = 'relu'))  
model_ann.add(Dropout(0.5))
model_ann.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
model_ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                1300      
                                                                 
 dense_1 (Dense)             (None, 25)                650       
                                                                 
 dropout (Dropout)           (None, 25)                0         
                                                                 
 dense_2 (Dense)             (None, 25)                650       
                                                                 
 dropout_1 (Dropout)         (None, 25)                0         
                                                                 
 dense_3 (Dense)             (None, 25)                650       
                                                                 
 dropout_2 (Dropout)         (None, 25)                0

In [ ]:
model_ann.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics='accuracy')
hist = model_ann.fit(X_train, y_train, epochs=100, batch_size = 50)

Epoch 1/100
29/29 [==============================] - 3s 3ms/step - loss: 0.6843 - accuracy: 0.5573
Epoch 2/100
29/29 [==============================] - 0s 3ms/step - loss: 0.6660 - accuracy: 0.6178
Epoch 3/100
29/29 [==============================] - 0s 3ms/step - loss: 0.6429 - accuracy: 0.6421
Epoch 4/100
29/29 [==============================] - 0s 3ms/step - loss: 0.6023 - accuracy: 0.6713
Epoch 5/100
29/29 [==============================] - 0s 3ms/step - loss: 0.5511 - accuracy: 0.7137
Epoch 6/100
29/29 [==============================] - 0s 3ms/step - loss: 0.4957 - accuracy: 0.7630
Epoch 7/100
29/29 [==============================] - 0s 3ms/step - loss: 0.4518 - accuracy: 0.7867
Epoch 8/100
29/29 [==============================] - 0s 3ms/step - loss: 0.4047 - accuracy: 0.8235
Epoch 9/100
29/29 [==============================] - 0s 3ms/step - loss: 0.3803 - accuracy: 0.8346
Epoch 10/100
29/29 [==============================] - 0s 3ms/step - loss: 0.3324 - accuracy: 0.8513
Epoch 11/

In [ ]:
y_pred_ann = model_ann.predict(X_test)
y_pred_ann = (y_pred_ann > 0.5)

In [ ]:
print(f'Accuracy: {100*accuracy_score(y_pred_ann, y_test): .3f}%')
print('\n')
print('Confusion Matrix: \n', confusion_matrix(y_pred_ann, y_test))
print('\n')
print('Classification Report: \n', classification_report(y_pred_ann, y_test))

Accuracy:  76.087%


Confusion Matrix: 
 [[243  22]
 [ 66  37]]


Classification Report: 
               precision    recall  f1-score   support

       False       0.79      0.92      0.85       265
        True       0.63      0.36      0.46       103

    accuracy                           0.76       368
   macro avg       0.71      0.64      0.65       368
weighted avg       0.74      0.76      0.74       368



We can choose Random Forest Classifier and ANN to next stage : Hyper Parameter Tuning.

## Hyper Parameter Tuning

### Random Forest

In [ ]:
model_rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
from sklearn.model_selection import GridSearchCV

model_rf_gs = RandomForestClassifier()

param_grid = {
    'bootstrap': [True, False],
    'max_depth': [None, 90, 100, 110],
    'max_features': [2, 3, 'auto','sqrt'],
    'min_samples_leaf': [1, 3, 5],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [100, 300, 500]
}

grid_search_rf = GridSearchCV(estimator = model_rf_gs, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search_rf.fit(X_train,y_train)

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [None, 90, 100, 110],
                         'max_features': [2, 3, 'auto', 'sqrt'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 300, 500]},
             verbose=2)

In [ ]:
grid_search_rf.best_params_

{'bootstrap': False,
 'max_depth': 100,
 'max_features': 2,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 300}

In [ ]:
model_rf_bg = grid_search_rf.best_estimator_
y_pred_rf_bg = model_rf_bg.predict(X_test)

In [ ]:
print(f'Accuracy: {100*accuracy_score(y_pred_rf_bg, y_test): .3f}%')

Accuracy:  83.967%


### Artificial Neural Network

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier(loss, optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 25, activation = 'relu', input_shape = (51,)))
    classifier.add(Dense(units = 25, activation = 'relu'))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(units = 25, activation = 'relu'))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(units = 25, activation = 'relu'))
    classifier.add(Dropout(0.5))
    classifier.add(Dense(units = 1, activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])
    return classifier


model_ann_gs = KerasClassifier(build_fn = build_classifier)

param_grid1 = {'batch_size': [20,30,50], 'epochs': [50,100,200],'loss': ['binary_crossentropy', 'mean_squared_error'], 'optimizer' : ['adam', 'rmsprop']}

In [ ]:
grid_search_ann = GridSearchCV(estimator = model_ann_gs, param_grid = param_grid1, cv = 3, n_jobs = -1, verbose = 2)

grid_search_ann.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Epoch 1/200
29/29 [==============================] - 1s 3ms/step - loss: 0.6946 - accuracy: 0.5608
Epoch 2/200
29/29 [==============================] - 0s 3ms/step - loss: 0.6597 - accuracy: 0.5997
Epoch 3/200
29/29 [==============================] - 0s 3ms/step - loss: 0.6363 - accuracy: 0.6136
Epoch 4/200
29/29 [==============================] - 0s 3ms/step - loss: 0.6100 - accuracy: 0.6192
Epoch 5/200
29/29 [==============================] - 0s 3ms/step - loss: 0.5927 - accuracy: 0.6220
Epoch 6/200
29/29 [==============================] - 0s 3ms/step - loss: 0.5586 - accuracy: 0.6275
Epoch 7/200
29/29 [==============================] - 0s 3ms/step - loss: 0.5235 - accuracy: 0.6400
Epoch 8/200
29/29 [==============================] - 0s 3ms/step - loss: 0.4925 - accuracy: 0.7067
Epoch 9/200
29/29 [==============================] - 0s 3ms/step - loss: 0.4872 - accuracy: 0.7797
Epoch 10/200
29/29 [===========================

GridSearchCV(cv=3,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f0793c6ab10>,
             n_jobs=-1,
             param_grid={'batch_size': [20, 30, 50], 'epochs': [50, 100, 200],
                         'loss': ['binary_crossentropy', 'mean_squared_error'],
                         'optimizer': ['adam', 'rmsprop']},
             verbose=2)

In [ ]:
grid_search_ann.best_params_

{'batch_size': 50,
 'epochs': 200,
 'loss': 'binary_crossentropy',
 'optimizer': 'adam'}

In [ ]:
model_ann_bg = grid_search_ann.best_estimator_

y_pred_ann_bg = model_ann_bg.predict(X_test)

print(f'Accuracy: {100*accuracy_score(y_pred_ann_bg, y_test): .3f}%')

Accuracy:  70.652%


## Model Evaluation

**Random Forest Classifer** 
*   Accuracy Before HPT : 79.620%
*   Accuracy After HPT : 83.967%


**Ariticial Neural Network**
* Accuracy Before HPT : 76.087%
* Accuracy After HPT : 70.652%

Random Forest Classifier after hyperparameter tuning will be our final model.

In [49]:
import pickle
pickle.dump(model_rf_bg, open('model/model_rf_final.pickle', 'wb'))